# Working with DBs - Notebook

## Objectives
- Understand Chicago datasets
- Load the datasets into tables in a SQLite database
- Execute SQL queries to answer assignment questions

## Understand the datasets

1. **Socioeconomic Indicators in Chicago**: Contains six socioeconomic indicators of public health significance and a "hardship index", for each community area of chicago, for the years 2008 - 2012
2. **Chicago Public Schools**: Shows all school level performance data used to create CPS School Report Cards for the 2021-2012 school year
3. **Chicago Crime Data**: Reflects reported incidents of crime (with the exception of murders where data exists for each victim) that occurred in the City of Chicago from 2001 to 2018

## Set Up

In [ ]:
%pip install ipython-sql prettytable pandas
%load_ext sql
import prettytable

prettytable.DEFAULT = "DEFAULT"

## Storing data sets in db

In [5]:
import sqlite3
import pandas as pd

# connection
con = sqlite3.connect("FinalDB.db")
cur = con.cursor()

%sql sqlite:///FinalDB.db

In [7]:
# storing dataset 1: Chicago Socioeconomic Indicators
table_ds_1 = "CENSUS_DATA"
df_se_indic = pd.read_csv("datasets/ChicagoCensusData.csv")
df_se_indic.to_sql(table_ds_1, con, if_exists="replace", index=False, method="multi")

# storing dataset 2: Chicago Public Schools
table_ds_2 = "CHICAGO_PUBLIC_SCHOOLS"
df_pub_sch = pd.read_csv("datasets/ChicagoPublicSchools.csv")
df_pub_sch.to_sql(table_ds_2, con, if_exists="replace", index=False, method="multi")

# storing dataset 3: Chicago Crime data
table_ds_3 = "CHICAGO_CRIME_DATA"
df_crime = pd.read_csv("datasets/ChicagoCrimeData.csv")
df_crime.to_sql(table_ds_3, con, if_exists="replace", index=False, method="multi")

533

## Exploring Data

In [17]:
%sql SELECT * FROM CHICAGO_CRIME_DATA ORDER BY DATE ASC limit 1

 * sqlite:///FinalDB.db
Done.


ID,CASE_NUMBER,DATE,BLOCK,IUCR,PRIMARY_TYPE,DESCRIPTION,LOCATION_DESCRIPTION,ARREST,DOMESTIC,BEAT,DISTRICT,WARD,COMMUNITY_AREA_NUMBER,FBICODE,X_COORDINATE,Y_COORDINATE,YEAR,LATITUDE,LONGITUDE,LOCATION
1326195,G021609,2001-01-11,087XX S ESCANABA AV,9901,DOMESTIC VIOLENCE,DOMESTIC VIOLENCE,APARTMENT,1,1,423,4,None,None,08B,1196869.0,1847416.0,2001,41.73617608,-87.55431961,"(41.73617608, -87.554319607)"


## Exercises
- Problem 1: Total number of crimes recorded in the CRIME table
- Problem 2: List community area names and numbers with per capita income less than 11000
- Problem 3: List all case numbers for crimes involving minors? (children are not considered minors for the purposes of crime analysis)
- Problem 4: List all kidnapping crimes involving child
- Problem 5: List the kind of crimes that were recorded at schools (No repetitions)
- Problem 6: List the type of schools along with the average safety score for each type
- Problem 7: List 5 community areas with highest % of households below poverty line
- Problem 8: Which Community area is most crime prone? Display the community area number only
- Problem 9: Use a subquery to find the name of the community area with highest hardships index
- Problem 10: Use a subquery to determine the Community Area Name with most number of crimes

Tables: CHICAGO_CRIME_DATA, CHICAGO_PUBLIC_SCHOOLS, CENSUS_DATA

### Problem 1: Total number of crimes recorded in the CRIME table -> 533

In [40]:
%sql SELECT count(*) FROM CHICAGO_CRIME_DATA

 * sqlite:///FinalDB.db
Done.


count(*)
533


### Problem 2: List community area names and numbers with per capita income less than 11000 --> 4

In [39]:
%%sql 
SELECT community_area_number, community_area_name, per_capita_income 
FROM CENSUS_DATA 
WHERE per_capita_income < 11000

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,PER_CAPITA_INCOME
26.0,West Garfield Park,10934
30.0,South Lawndale,10402
37.0,Fuller Park,10432
54.0,Riverdale,8201


### Problem 3: List all case numbers for crimes involving minors? (children are not considered minors for the purposes of crime analysis) --> 2


In [50]:
%%sql
SELECT * FROM CHICAGO_CRIME_DATA WHERE DESCRIPTION LIKE "%MINOR%"

 * sqlite:///FinalDB.db
Done.


ID,CASE_NUMBER,DATE,BLOCK,IUCR,PRIMARY_TYPE,DESCRIPTION,LOCATION_DESCRIPTION,ARREST,DOMESTIC,BEAT,DISTRICT,WARD,COMMUNITY_AREA_NUMBER,FBICODE,X_COORDINATE,Y_COORDINATE,YEAR,LATITUDE,LONGITUDE,LOCATION
3987219,HL266884,2005-03-31,024XX N CLARK ST,2210,LIQUOR LAW VIOLATION,SELL/GIVE/DEL LIQUOR TO MINOR,CONVENIENCE STORE,1,0,2333,19,43.0,7.0,22,1172680.0,1916483.0,2005,41.92626872,-87.64089934,"(41.926268719, -87.640899336)"
3266814,HK238408,2004-03-13,093XX S STONY ISLAND AVE,2230,LIQUOR LAW VIOLATION,ILLEGAL CONSUMPTION BY MINOR,ALLEY,1,0,413,4,8.0,48.0,22,1188539.0,1843379.0,2004,41.72530099,-87.58496589,"(41.72530099, -87.584965887)"


### Problem 4: List all kidnapping crimes involving child --> 1

In [56]:
%%sql
SELECT * FROM CHICAGO_CRIME_DATA WHERE PRIMARY_TYPE = "KIDNAPPING" AND DESCRIPTION LIKE "%CHILD%"

 * sqlite:///FinalDB.db
Done.


ID,CASE_NUMBER,DATE,BLOCK,IUCR,PRIMARY_TYPE,DESCRIPTION,LOCATION_DESCRIPTION,ARREST,DOMESTIC,BEAT,DISTRICT,WARD,COMMUNITY_AREA_NUMBER,FBICODE,X_COORDINATE,Y_COORDINATE,YEAR,LATITUDE,LONGITUDE,LOCATION
5276766,HN144152,2007-01-26,050XX W VAN BUREN ST,1792,KIDNAPPING,CHILD ABDUCTION/STRANGER,STREET,0,0,1533,15,29.0,25.0,20,1143050.0,1897546.0,2007,41.87490841,-87.75024931,"(41.874908413, -87.750249307)"


### Problem 5: List the kind of crimes that were recorded at schools (No repetitions) --> 6 

In [61]:
%%sql
SELECT DISTINCT(PRIMARY_TYPE) FROM CHICAGO_CRIME_DATA WHERE LOCATION_DESCRIPTION LIKE "%SCHOOL%"

 * sqlite:///FinalDB.db
Done.


PRIMARY_TYPE
BATTERY
CRIMINAL DAMAGE
NARCOTICS
ASSAULT
CRIMINAL TRESPASS
PUBLIC PEACE VIOLATION


### Problem 6: List the type of schools along with the average safety score for each type

In [63]:
%%sql
SELECT "Elementary, Middle, or High School", avg("SAFETY_SCORE") 
FROM CHICAGO_PUBLIC_SCHOOLS 
GROUP BY "Elementary, Middle, or High School"

 * sqlite:///FinalDB.db
Done.


"Elementary, Middle, or High School","avg(""SAFETY_SCORE"")"
ES,49.52038369304557
HS,49.62352941176471
MS,48.0


### Problem 7: List 5 community areas with highest % of households below poverty line

In [68]:
%%sql
SELECT COMMUNITY_AREA_NAME, PERCENT_HOUSEHOLDS_BELOW_POVERTY 
FROM CENSUS_DATA
ORDER BY PERCENT_HOUSEHOLDS_BELOW_POVERTY DESC
LIMIT 5

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME,PERCENT_HOUSEHOLDS_BELOW_POVERTY
Riverdale,56.5
Fuller Park,51.2
Englewood,46.6
North Lawndale,43.1
East Garfield Park,42.4


### Problem 8: Which Community area is most crime prone? Display the community area number only --> 25.0

In [74]:
%%sql
SELECT COMMUNITY_AREA_NUMBER, COUNT(*) 
FROM CHICAGO_CRIME_DATA
WHERE COMMUNITY_AREA_NUMBER != "None"
GROUP BY COMMUNITY_AREA_NUMBER
ORDER BY COUNT(ID) DESC
LIMIT 3

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NUMBER,COUNT(*)
25.0,43
23.0,22
68.0,21


### Problem 9: Use a subquery to find the name of the community area with highest hardships index

In [78]:
%%sql
SELECT * FROM CENSUS_DATA
WHERE HARDSHIP_INDEX = (
    SELECT MAX(HARDSHIP_INDEX) FROM CENSUS_DATA
)

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,PERCENT_OF_HOUSING_CROWDED,PERCENT_HOUSEHOLDS_BELOW_POVERTY,PERCENT_AGED_16__UNEMPLOYED,PERCENT_AGED_25__WITHOUT_HIGH_SCHOOL_DIPLOMA,PERCENT_AGED_UNDER_18_OR_OVER_64,PER_CAPITA_INCOME,HARDSHIP_INDEX
54.0,Riverdale,5.8,56.5,34.6,27.5,51.5,8201,98.0


### Problem 10: Use a subquery to determine the Community Area Name with most number of crimes

In [75]:
%%sql
SELECT COMMUNITY_AREA_NUMBER, COMMUNITY_AREA_NAME
FROM CENSUS_DATA
WHERE COMMUNITY_AREA_NUMBER = (
    SELECT COMMUNITY_AREA_NUMBER
    FROM CHICAGO_CRIME_DATA
    WHERE COMMUNITY_AREA_NUMBER != "None"
    GROUP BY COMMUNITY_AREA_NUMBER
    ORDER BY COUNT(ID) DESC
    LIMIT 1
)

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME
25.0,Austin
